In [1]:
import pandas as pd
import numpy as np
import json
import re
pd.set_option('display.max_columns', None)

## 1. Load and Transformation of the Dataset

In [2]:
with open('../data/inputs/MLA_100k_checked_v3.jsonlines', 'r') as file:
    data = [json.loads(line) for line in file]

df1 = pd.json_normalize(data)
df1.columns = df1.columns.str.replace('.', '_')
df1.shape

#nested lista
#non_mercado_pago_payment_methods
#attributes
#variations

(100000, 56)

In [3]:
df1.head()

,warranty,sub_status,condition,deal_ids,base_price,non_mercado_pago_payment_methods,seller_id,variations,site_id,listing_type_id,price,attributes,buying_mode,tags,listing_source,parent_item_id,coverage_areas,category_id,descriptions,last_updated,international_delivery_mode,pictures,id,official_store_id,differential_pricing,accepts_mercadopago,original_price,currency_id,thumbnail,title,automatic_relist,date_created,secure_thumbnail,stop_time,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,sold_quantity,available_quantity,seller_address_country_name,seller_address_country_id,seller_address_state_name,seller_address_state_id,seller_address_city_name,seller_address_city_id,shipping_local_pick_up,shipping_methods,shipping_tags,shipping_free_shipping,shipping_mode,shipping_dimensions,shipping_free_methods
0,None,[],new,[],80.0,"[{'description': 'Transferencia bancaria', 'id...",8208882349,[],MLA,bronze,80.0,[],buy_it_now,[dragged_bids_and_visits],,MLA6553902747,[],MLA126406,[{'id': 'MLA4695330653-912855983'}],2015-09-05T20:42:58.000Z,none,"[{'size': '500x375', 'secure_url': 'https://a2...",MLA4695330653,NaN,None,True,NaN,ARS,http://mla-s1-p.mlstatic.com/5386-MLA469533065...,Auriculares Samsung Originales Manos Libres Ca...,False,2015-09-05T20:42:53.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1446669773000,active,None,NaN,None,1,1441485773000,http://articulo.mercadolibre.com.ar/MLA4695330...,0,1,Argentina,AR,Capital Federal,AR-C,San Cristóbal,TUxBQlNBTjkwNTZa,True,[],[],False,not_specified,None,NaN
1,NUESTRA REPUTACION,[],used,[],2650.0,"[{'description': 'Transferencia bancaria', 'id...",8141699488,[],MLA,silver,2650.0,[],buy_it_now,[],,MLA7727150374,[],MLA10267,[{'id': 'MLA7160447179-930764806'}],2015-09-26T18:08:34.000Z,none,"[{'size': '499x334', 'secure_url': 'https://a2...",MLA7160447179,NaN,None,True,NaN,ARS,http://mla-s1-p.mlstatic.com/23223-MLA71604471...,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,False,2015-09-26T18:08:30.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1448474910000,active,None,NaN,None,1,1443290910000,http://articulo.mercadolibre.com.ar/MLA7160447...,0,1,Argentina,AR,Capital Federal,AR-C,Buenos Aires,,True,[],[],False,me2,None,NaN
2,None,[],used,[],60.0,"[{'description': 'Transferencia bancaria', 'id...",8386096505,[],MLA,bronze,60.0,[],buy_it_now,[dragged_bids_and_visits],,MLA6561247998,[],MLA1227,[{'id': 'MLA7367189936-916478256'}],2015-09-09T23:57:10.000Z,none,"[{'size': '375x500', 'secure_url': 'https://a2...",MLA7367189936,NaN,None,True,NaN,ARS,http://mla-s1-p.mlstatic.com/22076-MLA73671899...,"Antigua Revista Billiken, N° 1826, Año 1954",False,2015-09-09T23:57:07.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1447027027000,active,None,NaN,None,1,1441843027000,http://articulo.mercadolibre.com.ar/MLA7367189...,0,1,Argentina,AR,Capital Federal,AR-C,Boedo,TUxBQkJPRTQ0OTRa,True,[],[],False,me2,None,NaN
3,None,[],new,[],580.0,"[{'description': 'Transferencia bancaria', 'id...",5377752182,[],MLA,silver,580.0,[],buy_it_now,[],,None,[],MLA86345,[{'id': 'MLA9191625553-932309698'}],2015-10-05T16:03:50.306Z,none,"[{'size': '441x423', 'secure_url': 'https://a2...",MLA9191625553,NaN,None,True,NaN,ARS,http://mla-s2-p.mlstatic.com/183901-MLA9191625...,Alarma Guardtex Gx412 Seguridad Para El Automo...,False,2015-09-28T18:47:56.000Z,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1449191596000,active,None,NaN,None,1,1443466076000,http://articulo.mercadolibre.com.ar/MLA9191625...,0,1,Argentina,AR,Capital Federal,AR-C,Floresta,TUxBQkZMTzg5MjFa,True,[],[],False,me2,None,NaN
4,MI REPUTACION.,[],used,[],30.0,"[{'description': 'Transferencia bancaria', 'id...",2938071313,[],MLA,bronze,30.0,[],buy_it_now,[dragged_bids_and_visits],,MLA3133256685,[],MLA41287,[{'id': 'MLA7787961817-902981678'}],2015-08-28T13:37:41.000Z,none,"[{'size': '375x500', 'secure_url': 'https://a2...",MLA7787961817,NaN,None,True,NaN,ARS,http://mla-s2-p.mlstatic.com/13595-MLA77879618...,Se

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 56 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   warranty                          39104 non-null   object 
 1   sub_status                        100000 non-null  object 
 2   condition                         100000 non-null  object 
 3   deal_ids                          100000 non-null  object 
 4   base_price                        100000 non-null  float64
 5   non_mercado_pago_payment_methods  100000 non-null  object 
 6   seller_id                         100000 non-null  int64  
 7   variations                        100000 non-null  object 
 8   site_id                           100000 non-null  object 
 9   listing_type_id                   100000 non-null  object 
 10  price                             100000 non-null  float64
 11  attributes                        100000 non-null  ob

Reading the dataset, note this things:

- There is an unique column that can work as an index. The column is **id**.
- There are some columns with JSON format. Those columns seller_address, shipping need to be unnested.
- Also are columns nested in a list format. Those columns non_mercado_pago_payment_methods, descriptions, pictures, attributes, variations, shipping_free_methods need to be unnested.
- Other columns like tags, sub_status, shipping_tags are with various values on a list. Those can be replicated on different fields.
- Other columns are different indexes, or very high cardinality features, or are hard to transform into usefull fields. Those columns will be removed in the develop of the exercise.

In [5]:
# Setting df1 as the index
df1 = df1.set_index('id')

### Evaluation: Which columns do I drop?

In [6]:
print(f'The cardinality of the site_id feature is: {df1['site_id'].nunique()}')
print(f'The cardinality of the listing_source feature is: {df1['listing_source'].nunique()}')
print(f'The cardinality of the international_delivery_mode feature is: {df1['international_delivery_mode'].nunique()}')
print(f'The cardinality of the differential_pricing feature is: {df1['differential_pricing'].nunique()}')
print(f'The cardinality of the permalink feature is: {df1['permalink'].nunique()}')

The cardinality of the site_id feature is: 1
The cardinality of the listing_source feature is: 1
The cardinality of the international_delivery_mode feature is: 1
The cardinality of the differential_pricing feature is: 0
The cardinality of the permalink feature is: 100000


In [7]:
# Proceed to drop those columns
df1 = df1.drop(
    [
        'site_id', 
        'listing_source', 
        'international_delivery_mode', 
        'differential_pricing', 
        'permalink',
        'descriptions'
    ]
    , axis=1
) 

In [8]:
df1.head(5)

,warranty,sub_status,condition,deal_ids,base_price,non_mercado_pago_payment_methods,seller_id,variations,listing_type_id,price,attributes,buying_mode,tags,parent_item_id,coverage_areas,category_id,last_updated,pictures,official_store_id,accepts_mercadopago,original_price,currency_id,thumbnail,title,automatic_relist,date_created,secure_thumbnail,stop_time,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,sold_quantity,available_quantity,seller_address_country_name,seller_address_country_id,seller_address_state_name,seller_address_state_id,seller_address_city_name,seller_address_city_id,shipping_local_pick_up,shipping_methods,shipping_tags,shipping_free_shipping,shipping_mode,shipping_dimensions,shipping_free_methods
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MLA4695330653,None,[],new,[],80.0,"[{'description': 'Transferencia bancaria', 'id...",8208882349,[],bronze,80.0,[],buy_it_now,[dragged_bids_and_visits],MLA6553902747,[],MLA126406,2015-09-05T20:42:58.000Z,"[{'size': '500x375', 'secure_url': 'https://a2...",NaN,True,NaN,ARS,http://mla-s1-p.mlstatic.com/5386-MLA469533065...,Auriculares Samsung Originales Manos Libres Ca...,False,2015-09-05T20:42:53.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1446669773000,active,None,NaN,None,1,1441485773000,0,1,Argentina,AR,Capital Federal,AR-C,San Cristóbal,TUxBQlNBTjkwNTZa,True,[],[],False,not_specified,None,NaN
MLA7160447179,NUESTRA REPUTACION,[],used,[],2650.0,"[{'description': 'Transferencia bancaria', 'id...",8141699488,[],silver,2650.0,[],buy_it_now,[],MLA7727150374,[],MLA10267,2015-09-26T18:08:34.000Z,"[{'size': '499x334', 'secure_url': 'https://a2...",NaN,True,NaN,ARS,http://mla-s1-p.mlstatic.com/23223-MLA71604471...,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,False,2015-09-26T18:08:30.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1448474910000,active,None,NaN,None,1,1443290910000,0,1,Argentina,AR,Capital Federal,AR-C,Buenos Aires,,True,[],[],False,me2,None,NaN
MLA7367189936,None,[],used,[],60.0,"[{'description': 'Transferencia bancaria', 'id...",8386096505,[],bronze,60.0,[],buy_it_now,[dragged_bids_and_visits],MLA6561247998,[],MLA1227,2015-09-09T23:57:10.000Z,"[{'size': '375x500', 'secure_url': 'https://a2...",NaN,True,NaN,ARS,http://mla-s1-p.mlstatic.com/22076-MLA73671899...,"Antigua Revista Billiken, N° 1826, Año 1954",False,2015-09-09T23:57:07.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1447027027000,active,None,NaN,None,1,1441843027000,0,1,Argentina,AR,Capital Federal,AR-C,Boedo,TUxBQkJPRTQ0OTRa,True,[],[],False,me2,None,NaN
MLA9191625553,None,[],new,[],580.0,"[{'description': 'Transferencia bancaria', 'id...",5377752182,[],silver,580.0,[],buy_it_now,[],None,[],MLA86345,2015-10-05T16:03:50.306Z,"[{'size': '441x423', 'secure_url': 'https://a2...",NaN,True,NaN,ARS,http://mla-s2-p.mlstatic.com/183901-MLA9191625...,Alarma Guardtex Gx412 Seguridad Para El Automo...,False,2015-09-28T18:47:56.000Z,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1449191596000,active,None,NaN,None,1,1443466076000,0,1,Argentina,AR,Capital Federal,AR-C,Floresta,TUxBQkZMTzg5MjFa,True,[],[],False,me2,None,NaN
MLA7787961817,MI REPUTACION.,[],used,[],30.0,"[{'description': 'Transferencia bancaria', 'id...",2938071313,[],bronze,30.0,[],buy_it_now,[dragged_bids_and_visits],MLA3133256685,[],MLA41287,2015-08-28T13:37:41.000Z,"[{'size': '375x500', 'secure_url': 'https://a2...",NaN,True,NaN,ARS,http://mla-s2-p.mlstatic.com/13595-MLA77879618...,Serenata - Jennifer Blake,False,2015-08-24T22:07:20.000Z,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1445638040000,active,None,NaN,None,1,1440454040000,0,1,Argentina,AR,Buenos Aires,AR-B,Tres de febrero,TUxBQ1RSRTMxODE5NA,True,[],[],False,not_specified,None,NaN


### 1.3. Dealing with nested files - List format

As mentioned early, some fields are nested in a list format. In the file **ml_utils.py** are some functions that can help with the descomposition of this fields. 

For this case, I'll be using the function **expand_nested_fields**.

In [9]:
import sys
sys.path.append('../python_scripts')
from ml_utils import ml

In [10]:
df1[[
    'non_mercado_pago_payment_methods', 
    'pictures', 
    'attributes', 
    'variations', 
    'shipping_free_methods'
]].head()

,non_mercado_pago_payment_methods,pictures,attributes,variations,shipping_free_methods
id,,,,,
MLA4695330653,"[{'description': 'Transferencia bancaria', 'id...","[{'size': '500x375', 'secure_url': 'https://a2...",[],[],NaN
MLA7160447179,"[{'description': 'Transferencia bancaria', 'id...","[{'size': '499x334', 'secure_url': 'https://a2...",[],[],NaN
MLA7367189936,"[{'description': 'Transferencia bancaria', 'id...","[{'size': '375x500', 'secure_url': 'https://a2...",[],[],NaN
MLA9191625553,"[{'description': 'Transferencia bancaria', 'id...","[{'size': '441x423', 'secure_url': 'https://a2...",[],[],NaN
MLA7787961817,"[{'description': 'Transferencia bancaria', 'id...","[{'size': '375x500', 'secure_url': 'https://a2...",[],[],NaN


In [11]:
# Confirmation: attributes has different nested values
df1.attributes.value_counts().head(3)

attributes
[]                                                                                                                                                                                                                                                                                                                          87575
[{'value_id': 'female', 'attribute_group_id': 'DFLT', 'name': 'Género', 'value_name': 'Mujer', 'attribute_group_name': 'Otros', 'id': 'GENDER'}, {'value_id': 'Season-All-Season', 'attribute_group_id': 'FIND', 'name': 'Season', 'value_name': 'All-Season', 'attribute_group_name': 'Ficha técnica', 'id': 'Season'}]     1161
[{'value_id': 'female', 'attribute_group_id': 'DFLT', 'name': 'Género', 'value_name': 'Mujer', 'attribute_group_name': 'Otros', 'id': 'GENDER'}, {'value_id': 'Season-All-Season', 'attribute_group_id': 'DFLT', 'name': 'Season', 'value_name': 'All-Season', 'attribute_group_name': 'Otros', 'id': 'Season'}]             1110
Name: count, dtype: int

In [12]:
# Confirmation: variations has different nested values
df1.variations.value_counts().head(3)

variations
[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [13]:
# Confirmation: shipping_free_methods has different nested values
df1.shipping_free_methods.value_counts().head(3)

shipping_free_methods
[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 73328}]     2955
[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 73330}]       40
[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 501145}]       6
Name: count, dtype: int64

In [14]:
df1 = ml.expand_nested_fields(
    df1, 
    [
        'non_mercado_pago_payment_methods', 
        'pictures', 
        'attributes', 
        'variations', 
        'shipping_free_methods'
    ]
)

In [15]:
df1[[
    "non_mercado_pago_payment_methods_id",
    "non_mercado_pago_payment_methods_description",
    "non_mercado_pago_payment_methods_type",
    "pictures_size",
    "pictures_id",
    "pictures_url",
    "pictures_quality",
    "pictures_secure_url",
    "pictures_max_size",
    "attributes_id",
    "attributes_value_name",
    "attributes_attribute_group_id",
    "attributes_name",
    "attributes_attribute_group_name",
    "attributes_value_id",
    "variations_id",
    "variations_attribute_combinations",
    "variations_price",
    "variations_picture_ids",
    "variations_seller_custom_field",
    "variations_available_quantity",
    "variations_sold_quantity",
    "shipping_free_methods_id",
    "shipping_free_methods_rule_value",
    "shipping_free_methods_rule_free_mode"
]].head(10)

,non_mercado_pago_payment_methods_id,non_mercado_pago_payment_methods_description,non_mercado_pago_payment_methods_type,pictures_size,pictures_id,pictures_url,pictures_quality,pictures_secure_url,pictures_max_size,attributes_id,attributes_value_name,attributes_attribute_group_id,attributes_name,attributes_attribute_group_name,attributes_value_id,variations_id,variations_attribute_combinations,variations_price,variations_picture_ids,variations_seller_custom_field,variations_available_quantity,variations_sold_quantity,shipping_free_methods_id,shipping_free_methods_rule_value,shipping_free_methods_rule_free_mode
id,,,,,,,,,,,,,,,,,,,,,,,,,
MLA4695330653,MLATB,Transferencia bancaria,G,500x375,5386-MLA4695330653_052013,http://mla-s1-p.mlstatic.com/5386-MLA469533065...,,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1200x900,None,None,None,None,None,None,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None
MLA7160447179,MLATB,Transferencia bancaria,G,499x334,23223-MLA7160447179_022015,http://mla-s1-p.mlstatic.com/23223-MLA71604471...,,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,951x637,None,None,None,None,None,None,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None
MLA7367189936,MLATB,Transferencia bancaria,G,375x500,22076-MLA7367189936_012015,http://mla-s1-p.mlstatic.com/22076-MLA73671899...,,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,900x1200,None,None,None,None,None,None,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None
MLA9191625553,MLATB,Transferencia bancaria,G,441x423,183901-MLA9191625553_092015,http://mla-s2-p.mlstatic.com/183901-MLA9191625...,,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,441x423,None,None,None,None,None,None,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None
MLA7787961817,MLATB,Transferencia bancaria,G,375x500,13595-MLA7787961817_1713,http://mla-s2-p.mlstatic.com/13595-MLA77879618...,,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,480x640,None,None,None,None,None,None,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None
MLA6542680143,MLATB,Transferencia bancaria,G,250x250,6207-MLA6542680143_3765,http://mla-s2-p.mlstatic.com/6207-MLA654268014...,,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,250x250,None,None,None,None,None,None,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None
MLA8224797916,None,None,None,500x375,472901-MLA8224797916_102015,http://mla-s1-p.mlstatic.com/472901-MLA8224797...,,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1200x900,None,None,None,None,None,None,9.742953e+09,"[{'value_id': '92012', 'name': 'Color Primario...",180.0,"[472901-MLA20442937232_102015, 509801-MLA20442...",None,1.0,0.0,NaN,None,None
MLA1395917367,MLATB,Transferencia bancaria,G,500x375,9874-MLA1395917367_122013,http://mla-s2-p.mlstatic.com/9874-MLA139591736...,,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1024x768,None,None,None,None,None,None,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None
MLA6054779321,MLAOT,Tarjeta de crédito,N,500x500,797901-MLA6054779321_092015,http://mla-s1-p.mlstatic.com/797901-MLA6054779...,,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,800x800,PART_NUMBER,37123,DFLT,Número de pieza,Otros,,NaN,None,NaN,None,None,NaN,NaN,73328.0,None,country


In [16]:
df1[[
    "non_mercado_pago_payment_methods_id",
    "non_mercado_pago_payment_methods_description",
    "non_mercado_pago_payment_methods_type",
    "pictures_size",
    "pictures_id",
    "pictures_url",
    "pictures_quality",
    "pictures_secure_url",
    "pictures_max_size",
    "attributes_id",
    "attributes_value_name",
    "attributes_attribute_group_id",
    "attributes_name",
    "attributes_attribute_group_name",
    "attributes_value_id",
    "variations_id",
    "variations_attribute_combinations",
    "variations_price",
    "variations_picture_ids",
    "variations_seller_custom_field",
    "variations_available_quantity",
    "variations_sold_quantity",
    "shipping_free_methods_id",
    "shipping_free_methods_rule_value",
    "shipping_free_methods_rule_free_mode"
]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, MLA4695330653 to MLA9372431786
Data columns (total 25 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   non_mercado_pago_payment_methods_id           69441 non-null  object 
 1   non_mercado_pago_payment_methods_description  69441 non-null  object 
 2   non_mercado_pago_payment_methods_type         69441 non-null  object 
 3   pictures_size                                 99211 non-null  object 
 4   pictures_id                                   99211 non-null  object 
 5   pictures_url                                  99211 non-null  object 
 6   pictures_quality                              99211 non-null  object 
 7   pictures_secure_url                           99211 non-null  object 
 8   pictures_max_size                             99211 non-null  object 
 9   attributes_id                                

In [17]:
df1.drop(columns=['shipping_free_methods_rule_value', 'variations_attribute_combinations'], inplace = True)

### 1.4. Dealing with list separated columns

As mentioned early, separated like a list. In the file **ml_utils.py** are some functions that can help with the descomposition of this fields. 

For this case, I'll be using the function **expand_list_fields**.

In [18]:
df1[['tags', 'sub_status', 'shipping_tags']].head()

,tags,sub_status,shipping_tags
id,,,
MLA4695330653,[dragged_bids_and_visits],[],[]
MLA7160447179,[],[],[]
MLA7367189936,[dragged_bids_and_visits],[],[]
MLA9191625553,[],[],[]
MLA7787961817,[dragged_bids_and_visits],[],[]


In [19]:
df1.tags.value_counts().head(3)

tags
[dragged_bids_and_visits]                            72551
[]                                                   24910
[good_quality_thumbnail, dragged_bids_and_visits]     1330
Name: count, dtype: int64

In [20]:
df1.sub_status.value_counts().head(3)

sub_status
[]             99014
[suspended]      966
[expired]         13
Name: count, dtype: int64

In [21]:
df1.shipping_tags.value_counts().head(3)

shipping_tags
[]                               99931
[relisted_with_modifications]       59
[free_method_upgraded]              10
Name: count, dtype: int64

In [22]:
df1 = ml.expand_list_fields(df1, ['tags', 'sub_status', 'shipping_tags'])

In [23]:
df1[[
    "tags_free_relist",
    "tags_good_quality_thumbnail",
    "tags_dragged_bids_and_visits",
    "tags_poor_quality_thumbnail",
    "tags_dragged_visits",
    "sub_status_deleted",
    "sub_status_suspended",
    "sub_status_expired",
    "shipping_tags_relisted_with_modifications",
    "shipping_tags_free_method_upgraded"
]].head()

,tags_free_relist,tags_good_quality_thumbnail,tags_dragged_bids_and_visits,tags_poor_quality_thumbnail,tags_dragged_visits,sub_status_deleted,sub_status_suspended,sub_status_expired,shipping_tags_relisted_with_modifications,shipping_tags_free_method_upgraded
id,,,,,,,,,,
MLA4695330653,no,no,yes,no,no,no,no,no,no,no
MLA7160447179,no,no,no,no,no,no,no,no,no,no
MLA7367189936,no,no,yes,no,no,no,no,no,no,no
MLA9191625553,no,no,no,no,no,no,no,no,no,no
MLA7787961817,no,no,yes,no,no,no,no,no,no,no


In [24]:
df1[[
    "tags_free_relist",
    "tags_good_quality_thumbnail",
    "tags_dragged_bids_and_visits",
    "tags_poor_quality_thumbnail",
    "tags_dragged_visits",
    "sub_status_deleted",
    "sub_status_suspended",
    "sub_status_expired",
    "shipping_tags_relisted_with_modifications",
    "shipping_tags_free_method_upgraded"
]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, MLA4695330653 to MLA9372431786
Data columns (total 10 columns):
 #   Column                                     Non-Null Count   Dtype 
---  ------                                     --------------   ----- 
 0   tags_free_relist                           100000 non-null  object
 1   tags_good_quality_thumbnail                100000 non-null  object
 2   tags_dragged_bids_and_visits               100000 non-null  object
 3   tags_poor_quality_thumbnail                100000 non-null  object
 4   tags_dragged_visits                        100000 non-null  object
 5   sub_status_deleted                         100000 non-null  object
 6   sub_status_suspended                       100000 non-null  object
 7   sub_status_expired                         100000 non-null  object
 8   shipping_tags_relisted_with_modifications  100000 non-null  object
 9   shipping_tags_free_method_upgraded         100000 non-null  object
dtypes: obj

### 1.5. Transforming flags

In this step is the replace of flags into yes/no values.

In [25]:
df1['official_store_id'] = df1['official_store_id'].apply(lambda x: 'yes' if pd.notnull(x) else 'no')
df1['video_id'] = df1['video_id'].apply(lambda x: 'yes' if pd.notnull(x) else 'no')
df1['catalog_product_id'] = df1['catalog_product_id'].apply(lambda x: 'yes' if pd.notnull(x) else 'no')
df1['deal_ids'] = df1['deal_ids'].apply(lambda x: 'yes' if pd.notnull(x) else 'no')
df1['parent_item_id'] = df1['parent_item_id'].apply(lambda x: 'yes' if pd.notnull(x) else 'no')
df1['thumbnail'] = df1['thumbnail'].apply(lambda x: 'yes' if pd.notnull(x) else 'no')
df1['secure_thumbnail'] = df1['secure_thumbnail'].apply(lambda x: 'yes' if pd.notnull(x) else 'no')
df1['seller_id'] = df1['seller_id'].apply(lambda x: 'yes' if pd.notnull(x) else 'no')

C:\Users\EJLL\AppData\Local\Temp\ipykernel_9544\1120696160.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  df1['deal_ids'] = df1['deal_ids'].apply(lambda x: 'yes' if pd.notnull(x) else 'no')


### 1.5. Transforming ids

Here's happening the agrupation of most frecuent values into a top10 and others selection.

In [26]:
df1.category_id.value_counts()

category_id
MLA1227      4593
MLA2044      1950
MLA41287      932
MLA3530       758
MLA2038       665
             ... 
MLA44255        1
MLA79386        1
MLA355620       1
MLA23467        1
MLA340375       1
Name: count, Length: 10907, dtype: int64

In [27]:
top10 = df1['category_id'].value_counts().head(10).index
df1['category_id'] = df1['category_id'].apply(lambda x: x if x in top10 else 'OTHERS')

In [28]:
top10 = df1['attributes_value_id'].value_counts().head(10).index
df1['attributes_value_id'] = df1['attributes_value_id'].apply(lambda x: x if x in top10 else 'OTHERS')

In [29]:
top10 = df1['seller_address_city_id'].value_counts().head(10).index
df1['seller_address_city_id'] = df1['seller_address_city_id'].apply(lambda x: x if x in top10 else 'OTHERS')

### 1.6. Field 'warranty'

Here is the standarization of the field warranty in a very generalized form.

In [30]:
df1.warranty.value_counts()

warranty
Sí                                                                       5682
Sin garantía                                                             5442
SI                                                                        592
6 meses                                                                   547
6 MESES                                                                   397
                                                                         ... 
Garantía de un año a partir de la fecha de entrega!                         1
Garantía 1 año. La misma no incluye daños por mal uso o accidentales.       1
SE PRUEBAN LOS JUEGOS AL RETIRAR                                            1
Mi garantía son los comentarios de mis compradores.                         1
Pueden variar matasellos                                                    1
Name: count, Length: 10264, dtype: int64

In [31]:
df1['warranty'] = df1['warranty'].apply(ml.normalize_text)

In [32]:
df1['warranty'] = np.where(
    df1['warranty'].isnull(), 
    'no', 
    np.where(
        df1['warranty'] == 'si',
        'yes',
        np.where(
            df1['warranty'].str.contains('sin', na=False),
            'no',
            np.where(
                df1['warranty'].str.contains('mes', na=False),
                'meses',
                np.where(
                    df1['warranty'].str.contains('comentarios', case=False, na=False),
                    'no',
                    np.where(
                        df1['warranty'].str.contains('garantia', case=False, na=False),
                        'yes',
                        'otros'  # Al final, si ninguna de las condiciones anteriores se cumple, asigna 'otros'
                    )
                )
            )
        )
    )
)


In [33]:
df1.warranty.value_counts()

warranty
otros    74831
yes      13534
no        6590
meses     5045
Name: count, dtype: int64

### 1.7. Final validations

In [34]:
df1.head(10)

,warranty,condition,deal_ids,base_price,seller_id,listing_type_id,price,buying_mode,parent_item_id,coverage_areas,category_id,last_updated,official_store_id,accepts_mercadopago,original_price,currency_id,thumbnail,title,automatic_relist,date_created,secure_thumbnail,stop_time,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,sold_quantity,available_quantity,seller_address_country_name,seller_address_country_id,seller_address_state_name,seller_address_state_id,seller_address_city_name,seller_address_city_id,shipping_local_pick_up,shipping_methods,shipping_free_shipping,shipping_mode,shipping_dimensions,non_mercado_pago_payment_methods_description,non_mercado_pago_payment_methods_type,non_mercado_pago_payment_methods_id,pictures_size,pictures_max_size,pictures_secure_url,pictures_url,pictures_quality,pictures_id,attributes_value_id,attributes_name,attributes_attribute_group_name,attributes_attribute_group_id,attributes_value_name,attributes_id,variations_price,variations_available_quantity,variations_picture_ids,variations_id,variations_sold_quantity,variations_seller_custom_field,shipping_free_methods_rule_free_mode,shipping_free_methods_id,tags_dragged_visits,tags_dragged_bids_and_visits,tags_poor_quality_thumbnail,tags_good_quality_thumbnail,tags_free_relist,sub_status_deleted,sub_status_expired,sub_status_suspended,shipping_tags_free_method_upgraded,shipping_tags_relisted_with_modifications
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MLA4695330653,otros,new,no,80.0,yes,bronze,80.0,buy_it_now,yes,[],OTHERS,2015-09-05T20:42:58.000Z,no,True,NaN,ARS,yes,Auriculares Samsung Originales Manos Libres Ca...,False,2015-09-05T20:42:53.000Z,yes,1446669773000,active,no,no,None,1,1441485773000,0,1,Argentina,AR,Capital Federal,AR-C,San Cristóbal,OTHERS,True,[],False,not_specified,None,Transferencia bancaria,G,MLATB,500x375,1200x900,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,http://mla-s1-p.mlstatic.com/5386-MLA469533065...,,5386-MLA4695330653_052013,OTHERS,None,None,None,None,None,NaN,NaN,None,NaN,NaN,None,None,NaN,no,yes,no,no,no,no,no,no,no,no
MLA7160447179,otros,used,no,2650.0,yes,silver,2650.0,buy_it_now,yes,[],OTHERS,2015-09-26T18:08:34.000Z,no,True,NaN,ARS,yes,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,False,2015-09-26T18:08:30.000Z,yes,1448474910000,active,no,no,None,1,1443290910000,0,1,Argentina,AR,Capital Federal,AR-C,Buenos Aires,,True,[],False,me2,None,Transferencia bancaria,G,MLATB,499x334,951x637,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,http://mla-s1-p.mlstatic.com/23223-MLA71604471...,,23223-MLA7160447179_022015,OTHERS,None,None,None,None,None,NaN,NaN,None,NaN,NaN,None,None,NaN,no,no,no,no,no,no,no,no,no,no
MLA7367189936,otros,used,no,60.0,yes,bronze,60.0,buy_it_now,yes,[],MLA1227,2015-09-09T23:57:10.000Z,no,True,NaN,ARS,yes,"Antigua Revista Billiken, N° 1826, Año 1954",False,2015-09-09T23:57:07.000Z,yes,1447027027000,active,no,no,None,1,1441843027000,0,1,Argentina,AR,Capital Federal,AR-C,Boedo,OTHERS,True,[],False,me2,None,Transferencia bancaria,G,MLATB,375x500,900x1200,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,http://mla-s1-p.mlstatic.com/22076-MLA73671899...,,22076-MLA7367189936_012015,OTHERS,None,None,None,None,None,NaN,NaN,None,NaN,NaN,None,None,NaN,no,yes,no,no,no,no,no,no,no,no
MLA9191625553,otros,new,no,580.0,yes,silver,580.0,buy_it_now,no,[],OTHERS,2015-10-05T16:03:50.306Z,no,True,NaN,ARS,yes,Alarma Guardtex Gx412 Seguridad Para El Automo...,False,2015-09-28T18:47:56.000Z,yes,1449191596000,active,no,no,None,1,1443466076000,0,1,Argentina,AR,Capital Federal,AR-C,Floresta,OTHERS,True,[],False,me2,None,Transferencia bancaria,G,MLATB,441x423,441x423,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,http://mla-s2-p.mlstatic.com/183901-MLA9191625...,,183901-MLA9191625553_092015,OTHERS,None,None,None,None,None,NaN,NaN,None,NaN,NaN,None,None,NaN,no,no,no,no,no,no,no,no,no,no
MLA7787961817,otros,used,no,30.0,yes,bronze,30.0,buy_it_now,yes,[],

In [35]:
df1.dtypes

warranty                                      object
condition                                     object
deal_ids                                      object
base_price                                   float64
seller_id                                     object
                                              ...   
sub_status_deleted                            object
sub_status_expired                            object
sub_status_suspended                          object
shipping_tags_free_method_upgraded            object
shipping_tags_relisted_with_modifications     object
Length: 74, dtype: object

In [36]:
# Transforming bools into yes/no to give all the variables the same treatments
bool_cols = df1.select_dtypes(include=['bool']).columns
df1[bool_cols] = df1[bool_cols].applymap(lambda x: 'yes' if x else 'no')

C:\Users\EJLL\AppData\Local\Temp\ipykernel_9544\3985679981.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1[bool_cols] = df1[bool_cols].applymap(lambda x: 'yes' if x else 'no')


In [37]:
# Replacing empty lists for np.nan
df1 = df1.replace('[]', np.nan)
df1 = df1.applymap(lambda x: np.nan if isinstance(x, list) and len(x) == 0 else x)
df1.head()

C:\Users\EJLL\AppData\Local\Temp\ipykernel_9544\3966019391.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = df1.applymap(lambda x: np.nan if isinstance(x, list) and len(x) == 0 else x)


,warranty,condition,deal_ids,base_price,seller_id,listing_type_id,price,buying_mode,parent_item_id,coverage_areas,category_id,last_updated,official_store_id,accepts_mercadopago,original_price,currency_id,thumbnail,title,automatic_relist,date_created,secure_thumbnail,stop_time,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,sold_quantity,available_quantity,seller_address_country_name,seller_address_country_id,seller_address_state_name,seller_address_state_id,seller_address_city_name,seller_address_city_id,shipping_local_pick_up,shipping_methods,shipping_free_shipping,shipping_mode,shipping_dimensions,non_mercado_pago_payment_methods_description,non_mercado_pago_payment_methods_type,non_mercado_pago_payment_methods_id,pictures_size,pictures_max_size,pictures_secure_url,pictures_url,pictures_quality,pictures_id,attributes_value_id,attributes_name,attributes_attribute_group_name,attributes_attribute_group_id,attributes_value_name,attributes_id,variations_price,variations_available_quantity,variations_picture_ids,variations_id,variations_sold_quantity,variations_seller_custom_field,shipping_free_methods_rule_free_mode,shipping_free_methods_id,tags_dragged_visits,tags_dragged_bids_and_visits,tags_poor_quality_thumbnail,tags_good_quality_thumbnail,tags_free_relist,sub_status_deleted,sub_status_expired,sub_status_suspended,shipping_tags_free_method_upgraded,shipping_tags_relisted_with_modifications
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MLA4695330653,otros,new,no,80.0,yes,bronze,80.0,buy_it_now,yes,NaN,OTHERS,2015-09-05T20:42:58.000Z,no,yes,NaN,ARS,yes,Auriculares Samsung Originales Manos Libres Ca...,no,2015-09-05T20:42:53.000Z,yes,1446669773000,active,no,no,None,1,1441485773000,0,1,Argentina,AR,Capital Federal,AR-C,San Cristóbal,OTHERS,yes,NaN,no,not_specified,None,Transferencia bancaria,G,MLATB,500x375,1200x900,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,http://mla-s1-p.mlstatic.com/5386-MLA469533065...,,5386-MLA4695330653_052013,OTHERS,None,None,None,None,None,NaN,NaN,None,NaN,NaN,None,None,NaN,no,yes,no,no,no,no,no,no,no,no
MLA7160447179,otros,used,no,2650.0,yes,silver,2650.0,buy_it_now,yes,NaN,OTHERS,2015-09-26T18:08:34.000Z,no,yes,NaN,ARS,yes,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,no,2015-09-26T18:08:30.000Z,yes,1448474910000,active,no,no,None,1,1443290910000,0,1,Argentina,AR,Capital Federal,AR-C,Buenos Aires,,yes,NaN,no,me2,None,Transferencia bancaria,G,MLATB,499x334,951x637,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,http://mla-s1-p.mlstatic.com/23223-MLA71604471...,,23223-MLA7160447179_022015,OTHERS,None,None,None,None,None,NaN,NaN,None,NaN,NaN,None,None,NaN,no,no,no,no,no,no,no,no,no,no
MLA7367189936,otros,used,no,60.0,yes,bronze,60.0,buy_it_now,yes,NaN,MLA1227,2015-09-09T23:57:10.000Z,no,yes,NaN,ARS,yes,"Antigua Revista Billiken, N° 1826, Año 1954",no,2015-09-09T23:57:07.000Z,yes,1447027027000,active,no,no,None,1,1441843027000,0,1,Argentina,AR,Capital Federal,AR-C,Boedo,OTHERS,yes,NaN,no,me2,None,Transferencia bancaria,G,MLATB,375x500,900x1200,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,http://mla-s1-p.mlstatic.com/22076-MLA73671899...,,22076-MLA7367189936_012015,OTHERS,None,None,None,None,None,NaN,NaN,None,NaN,NaN,None,None,NaN,no,yes,no,no,no,no,no,no,no,no
MLA9191625553,otros,new,no,580.0,yes,silver,580.0,buy_it_now,no,NaN,OTHERS,2015-10-05T16:03:50.306Z,no,yes,NaN,ARS,yes,Alarma Guardtex Gx412 Seguridad Para El Automo...,no,2015-09-28T18:47:56.000Z,yes,1449191596000,active,no,no,None,1,1443466076000,0,1,Argentina,AR,Capital Federal,AR-C,Floresta,OTHERS,yes,NaN,no,me2,None,Transferencia bancaria,G,MLATB,441x423,441x423,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,http://mla-s2-p.mlstatic.com/183901-MLA9191625...,,183901-MLA9191625553_092015,OTHERS,None,None,None,None,None,NaN,NaN,None,NaN,NaN,None,None,NaN,no,no,no,no,no,no,no,no,no,no
MLA7787961817,otros,used,no,30.0,yes,bronze,30.0,buy_it_now,yes,NaN,MLA41287,2015-08-28T13:

In [38]:
#-Droping - Argument: irrelevant or very hard to understand information
df1 = df1.drop(
    columns = [ 
        'seller_address_country_id', 
        'seller_address_state_name', 
        'title', 
        'pictures_url', 
        'pictures_secure_url', 
        'pictures_id',
        'pictures_size',
        'pictures_max_size',
        'variations_picture_ids',
        'seller_address_city_name',
        'variations_seller_custom_field',
        'attributes_id'
    ]
)

In [39]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, MLA4695330653 to MLA9372431786
Data columns (total 62 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   warranty                                      100000 non-null  object 
 1   condition                                     100000 non-null  object 
 2   deal_ids                                      100000 non-null  object 
 3   base_price                                    100000 non-null  float64
 4   seller_id                                     100000 non-null  object 
 5   listing_type_id                               100000 non-null  object 
 6   price                                         100000 non-null  float64
 7   buying_mode                                   100000 non-null  object 
 8   parent_item_id                                100000 non-null  object 
 9   coverage_areas                    

In [40]:
df1 = df1.dropna(axis=1, how='all')

### 1.8. Treatment to the timestamps

In [41]:
df1['last_updated'] = pd.to_datetime(df1['last_updated'])
df1['date_created'] = pd.to_datetime(df1['date_created'])

In [42]:
df1

,warranty,condition,deal_ids,base_price,seller_id,listing_type_id,price,buying_mode,parent_item_id,category_id,last_updated,official_store_id,accepts_mercadopago,original_price,currency_id,thumbnail,automatic_relist,date_created,secure_thumbnail,stop_time,status,video_id,catalog_product_id,initial_quantity,start_time,sold_quantity,available_quantity,seller_address_country_name,seller_address_state_id,seller_address_city_id,shipping_local_pick_up,shipping_free_shipping,shipping_mode,shipping_dimensions,non_mercado_pago_payment_methods_description,non_mercado_pago_payment_methods_type,non_mercado_pago_payment_methods_id,pictures_quality,attributes_value_id,attributes_name,attributes_attribute_group_name,attributes_attribute_group_id,attributes_value_name,variations_price,variations_available_quantity,variations_id,variations_sold_quantity,shipping_free_methods_rule_free_mode,shipping_free_methods_id,tags_dragged_visits,tags_dragged_bids_and_visits,tags_poor_quality_thumbnail,tags_good_quality_thumbnail,tags_free_relist,sub_status_deleted,sub_status_expired,sub_status_suspended,shipping_tags_free_method_upgraded,shipping_tags_relisted_with_modifications
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MLA4695330653,otros,new,no,80.0,yes,bronze,80.0,buy_it_now,yes,OTHERS,2015-09-05 20:42:58+00:00,no,yes,NaN,ARS,yes,no,2015-09-05 20:42:53+00:00,yes,1446669773000,active,no,no,1,1441485773000,0,1,Argentina,AR-C,OTHERS,yes,no,not_specified,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,yes,no,no,no,no,no,no,no,no
MLA7160447179,otros,used,no,2650.0,yes,silver,2650.0,buy_it_now,yes,OTHERS,2015-09-26 18:08:34+00:00,no,yes,NaN,ARS,yes,no,2015-09-26 18:08:30+00:00,yes,1448474910000,active,no,no,1,1443290910000,0,1,Argentina,AR-C,,yes,no,me2,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,no,no,no,no,no,no,no,no,no
MLA7367189936,otros,used,no,60.0,yes,bronze,60.0,buy_it_now,yes,MLA1227,2015-09-09 23:57:10+00:00,no,yes,NaN,ARS,yes,no,2015-09-09 23:57:07+00:00,yes,1447027027000,active,no,no,1,1441843027000,0,1,Argentina,AR-C,OTHERS,yes,no,me2,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,yes,no,no,no,no,no,no,no,no
MLA9191625553,otros,new,no,580.0,yes,silver,580.0,buy_it_now,no,OTHERS,2015-10-05 16:03:50.306000+00:00,no,yes,NaN,ARS,yes,no,2015-09-28 18:47:56+00:00,yes,1449191596000,active,no,no,1,1443466076000,0,1,Argentina,AR-C,OTHERS,yes,no,me2,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,no,no,no,no,no,no,no,no,no
MLA7787961817,otros,used,no,30.0,yes,bronze,30.0,buy_it_now,yes,MLA41287,2015-08-28 13:37:41+00:00,no,yes,NaN,ARS,yes,no,2015-08-24 22:07:20+00:00,yes,1445638040000,active,no,no,1,1440454040000,0,1,Argentina,AR-B,OTHERS,yes,no,not_specified,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,yes,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MLA7015262230,otros,used,no,30.0,yes,free,30.0,buy_it_now,yes,MLA41287,2015-09-28 01:05:00+00:00,no,yes,NaN,ARS,yes,no,2015-09-28 01:04:49+00:00,yes,1448586289000,active,no,no,1,1443402289000,0,1,Argentina,AR-B,,yes,no,me2,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,yes,no,no,no,no,no,no,no,no
MLA6831230402,otros,new,no,239.0,yes,bronze,239.0,buy_it_now,yes,OTHERS,2015-09-11 18:09:04+00:00,no,yes,NaN,ARS,yes,no,2015-09-11 18:09:02+00:00,yes,1447178942000,active,no,no,5,1441994942000,0,5,Argentina,AR-B,OTHERS,no,no,not_specified,None,Tarjeta de crédito,N,MLAOT,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,no,no,no,no,no,no,no,no,no
MLA4632330570,otros,used,no,600.0,yes,bronze,600.0,buy_it_now,

In [43]:
df1 = ml.expand_datetime_columns(df1, ['last_updated', 'date_created', 'stop_time', 'start_time'])

In [44]:
df1.head()

,warranty,condition,deal_ids,base_price,seller_id,listing_type_id,price,buying_mode,parent_item_id,category_id,official_store_id,accepts_mercadopago,original_price,currency_id,thumbnail,automatic_relist,secure_thumbnail,status,video_id,catalog_product_id,initial_quantity,sold_quantity,available_quantity,seller_address_country_name,seller_address_state_id,seller_address_city_id,shipping_local_pick_up,shipping_free_shipping,shipping_mode,shipping_dimensions,non_mercado_pago_payment_methods_description,non_mercado_pago_payment_methods_type,non_mercado_pago_payment_methods_id,pictures_quality,attributes_value_id,attributes_name,attributes_attribute_group_name,attributes_attribute_group_id,attributes_value_name,variations_price,variations_available_quantity,variations_id,variations_sold_quantity,shipping_free_methods_rule_free_mode,shipping_free_methods_id,tags_dragged_visits,tags_dragged_bids_and_visits,tags_poor_quality_thumbnail,tags_good_quality_thumbnail,tags_free_relist,sub_status_deleted,sub_status_expired,sub_status_suspended,shipping_tags_free_method_upgraded,shipping_tags_relisted_with_modifications,last_updated_year,last_updated_month,last_updated_day,last_updated_hour,last_updated_minute,last_updated_second,date_created_year,date_created_month,date_created_day,date_created_hour,date_created_minute,date_created_second,stop_time_year,stop_time_month,stop_time_day,stop_time_hour,stop_time_minute,stop_time_second,start_time_year,start_time_month,start_time_day,start_time_hour,start_time_minute,start_time_second
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MLA4695330653,otros,new,no,80.0,yes,bronze,80.0,buy_it_now,yes,OTHERS,no,yes,NaN,ARS,yes,no,yes,active,no,no,1,0,1,Argentina,AR-C,OTHERS,yes,no,not_specified,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,yes,no,no,no,no,no,no,no,no,2015,9,5,20,42,58,2015,9,5,20,42,53,1970,1,1,0,24,6,1970,1,1,0,24,1
MLA7160447179,otros,used,no,2650.0,yes,silver,2650.0,buy_it_now,yes,OTHERS,no,yes,NaN,ARS,yes,no,yes,active,no,no,1,0,1,Argentina,AR-C,,yes,no,me2,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,no,no,no,no,no,no,no,no,no,2015,9,26,18,8,34,2015,9,26,18,8,30,1970,1,1,0,24,8,1970,1,1,0,24,3
MLA7367189936,otros,used,no,60.0,yes,bronze,60.0,buy_it_now,yes,MLA1227,no,yes,NaN,ARS,yes,no,yes,active,no,no,1,0,1,Argentina,AR-C,OTHERS,yes,no,me2,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,yes,no,no,no,no,no,no,no,no,2015,9,9,23,57,10,2015,9,9,23,57,7,1970,1,1,0,24,7,1970,1,1,0,24,1
MLA9191625553,otros,new,no,580.0,yes,silver,580.0,buy_it_now,no,OTHERS,no,yes,NaN,ARS,yes,no,yes,active,no,no,1,0,1,Argentina,AR-C,OTHERS,yes,no,me2,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,no,no,no,no,no,no,no,no,no,2015,10,5,16,3,50,2015,9,28,18,47,56,1970,1,1,0,24,9,1970,1,1,0,24,3
MLA7787961817,otros,used,no,30.0,yes,bronze,30.0,buy_it_now,yes,MLA41287,no,yes,NaN,ARS,yes,no,yes,active,no,no,1,0,1,Argentina,AR-B,OTHERS,yes,no,not_specified,None,Transferencia bancaria,G,MLATB,,OTHERS,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,no,yes,no,no,no,no,no,no,no,no,2015,8,28,13,37,41,2015,8,24,22,7,20,1970,1,1,0,24,5,1970,1,1,0,24,0


### 1.9. Dropping no cardinality features and high cardinality features

In [45]:
for col in df1.columns:
    if df1[col].apply(type).nunique() > 1:
        df1[col] = df1[col].astype(object)

In [46]:
cols_to_drop = [col for col in df1.columns if df1[col].nunique() == 1]
df1 = df1.drop(columns=cols_to_drop)

In [47]:
threshold = 300
cat_columns = df1.select_dtypes(include=['object']).columns
cols_to_drop = [col for col in cat_columns if df1[col].nunique() > threshold]
df1.drop(columns=cols_to_drop, inplace=True)

### 1.10. Checking high correlations

In [48]:
df1[['base_price', 'price']].corr()

,base_price,price
base_price,1.0,1.0
price,1.0,1.0


In [49]:
df1[['start_time_minute', 'start_time_second']].corr()

,start_time_minute,start_time_second
start_time_minute,1.000000,-0.989678
start_time_second,-0.989678,1.000000


In [50]:
df1[['initial_quantity', 'available_quantity']].corr()

,initial_quantity,available_quantity
initial_quantity,1.000000,0.999948
available_quantity,0.999948,1.000000


In [51]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, MLA4695330653 to MLA9372431786
Data columns (total 65 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   warranty                                      100000 non-null  object 
 1   condition                                     100000 non-null  object 
 2   deal_ids                                      100000 non-null  object 
 3   base_price                                    100000 non-null  float64
 4   listing_type_id                               100000 non-null  object 
 5   price                                         100000 non-null  float64
 6   buying_mode                                   100000 non-null  object 
 7   parent_item_id                                100000 non-null  object 
 8   category_id                                   100000 non-null  object 
 9   official_store_id                 

In [52]:
df1.drop(
    columns=['price', 'start_time_minute', 'available_quantity'],
    inplace=True
)

### 1.10. Exporting parquet

In [53]:
categorical_cols = df1.select_dtypes(include=['object', 'category']).columns

for col in categorical_cols:
    df1[col] = df1[col].apply(ml.normalize_text)

In [54]:
df1 = df1.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [55]:
df1.to_parquet('../data/staging/final_df.parquet')